In [1]:
from snowflake.snowpark import Session


connection_parameters = {

     "account":"zfb96811.us-east-1", 
     "role":"ACCOUNTADMIN", 
     "database":"ANALYTICS", 
     "schema":"PUBLIC",
     "warehouse":"COMPUTE_WH",
    "user": "IMAHAKUMARA",
    "password":"gEfnun-dewnin-0nadpy"    
}

session = Session.builder.configs(connection_parameters).create()

In [3]:
import os
import yaml
from pathlib import Path

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss, recall_score, confusion_matrix


from snowflake.snowpark.functions import col
import snowflake.snowpark.functions as f

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/oceansxyz/predictive_churn/.venv/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <89AD948E-E564-3266-867D-7AF89D6488F0> /Users/oceansxyz/predictive_churn/.venv/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libom

In [ ]:
#dataloader

df = session.table("analytics.analytics_inference.bimonthly_churn_features_1")

df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SNAPSHOT_WEEK"  |"SUBSCRIPTIONID"                      |"EMAIL"                    |"CREATEDAT"  |"CANCELLEDAT"  |"CANCELLED_WEEK"  |"SNAPSHOT_WEEK_PLUS_2"  |"LIFETIME"  |"CHURN_LABEL_14_DAY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2025-04-14       |d20029ba-b73a-4ac4-888f-3ad592312e36  |fishinchick74@gmail.com    |2024-10-24   |NULL           |NULL              |2025-04-28              |25          |0                     |
|2025-04-14       |0b3062fc-4644-4be4-b320-4b57ec5b4cd0  |dianetaylor@me.com         |2025-02-21   |2025-04-28     |2025-04-28        |2025-04-28              |10          |1                     |
|2025-04-14    

In [ ]:


# Assume you're in notebooks/
notebook_dir = Path().resolve()
root = notebook_dir.parent
config_path = root / "configs" / "bimonthly.yaml"

with config_path.open("r") as f:
    config = yaml.safe_load(f)

features = config['features']
label = config['label']

In [12]:
df_pandas = df.to_pandas()
# Then select features and label
X = df_pandas[features]
y = df_pandas[label]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
model = xgb.XGBClassifier(use_label_encoder=False, 
                          eval_metric='logloss',
                          n_estimators=500,
                          eta=0.05,
                          scale_pos_weight=42,
                          #enable_categorical=True
                         )

model.fit(X_train, y_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.05, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, ...)